In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import load_img,img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import shutil
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
image_path='/kaggle/input/footwearr/dataset/training'
#Define the vgg feature extractor.
vgg_model=VGG16(weights='imagenet',include_top=False)

def extract_features(image_path):
    img=image.load_img(image_path,target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    predictions = vgg_model.predict(x)
    predictions=predictions.flatten()
    
    return predictions


In [ ]:
#Create lists of features and paths.
features=[]
images_list=os.listdir('/kaggle/input/footwearr/dataset/training/shoe boot sandal')
images_list=list(map(lambda x:'/kaggle/input/footwearr/dataset/training/shoe boot sandal/'+x,images_list))

for i in tqdm(range(len(images_list))):
    filepath=images_list[i]
    img_features=extract_features(filepath)
    features.append(img_features)

In [ ]:
from sklearn import manifold, decomposition
features_array = np.array(features)
print("Shape of the features before the application of the PCA transformation {}".format(features_array.shape))
pca = decomposition.PCA(n_components=0.9)
reduc_features = pca.fit_transform(features_array)
print("Shape of the features after PCA transformation {}".format(reduc_features.shape))

In [ ]:
k = 3
clusters = KMeans(k)
clusters.fit(reduc_features)

In [ ]:
df_results = pd.DataFrame(images_list,columns=['image_file'])
df_results["Cluster_label"] = clusters.labels_
df_results.sample(10)

In [ ]:
df_results.head()
os.makedirs('/kaggle/working/image1')
os.makedirs('/kaggle/working/image2')
os.makedirs('/kaggle/working/image3')
path1='/kaggle/working/image1'
path2='/kaggle/working/image2'
path3='/kaggle/working/image3'


In [ ]:
for index,row in df_results.iterrows():
    if row['Cluster_label']==0:
        shutil.copy(row['image_file'], '/kaggle/working/image1')
    elif row["Cluster_label"]==1:
        shutil.copy(row['image_file'], '/kaggle/working/image2')
    else:
        shutil.copy(row['image_file'], '/kaggle/working/image3')

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = '/kaggle/working/', file_name = 'directory.zip'):
     os.chdir(directory)
     zip_ref = zipfile.ZipFile(file_name, mode='w')
     for folder, _, files in os.walk(directory):
         for file in files:
             if file_name in file:
                 pass
             else:
                 zip_ref.write(os.path.join(folder, file))

     return FileLink(file_name)

In [ ]:
zip_dir()